In [65]:
#!pip install rtmlib==0.0.13
#!pip install opencv-python==4.11.0.86
#!pip install librosa==0.11.0
#!pip install Cython==3.0.12
# Microsoft Visual C++ 14.0 or greater is required.
# Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
# The pip version of madmon is outdated and the github repo has many changes recently
#!pip uninstall -y madmom
#!conda install git+https://github.com/CPJKU/madmom.git
#!cd madmom && python setup.py develop --user
#!pip install pyaudio==0.2.14
#!pip install beatnet==1.1.1
#!pip install soundfile==0.13.1
# Install FFmpeg in Windows
# Source: https://phoenixnap.com/kb/ffmpeg-windows
#!pip cache purge
#!pip install moviepy==2.1.1
#!pip install onnxruntime==1.21.1

In [1]:
import os

move_filename = '022dccf'
moves_filepath = os.path.join('outputs', 'moves.json')

In [5]:
!ffmpeg -i inputs/videos/022dccf.mp4 -ab 160k -ac 2 -ar 44100 -vn processed/videos/022dccf.wav

^C


In [7]:
from BeatNet.BeatNet import BeatNet
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
audio_filepath = os.path.join('processed', 'videos', f'{move_filename}.wav')
estimator = BeatNet(1, mode='offline', inference_model='DBN', plot=[], thread=False)
beats = estimator.process(audio_filepath)
beats

C:\Users\user\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\user\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\user\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


array([[0.22, 4.  ],
       [0.66, 1.  ],
       [1.14, 2.  ],
       [1.58, 3.  ],
       [2.04, 4.  ],
       [2.52, 1.  ],
       [2.98, 2.  ],
       [3.44, 3.  ],
       [3.9 , 4.  ],
       [4.38, 1.  ],
       [4.84, 2.  ],
       [5.3 , 3.  ],
       [5.78, 4.  ],
       [6.24, 1.  ],
       [6.7 , 2.  ],
       [7.16, 3.  ],
       [7.62, 4.  ],
       [8.08, 1.  ]])

In [17]:
# 9da1eb8 = 4
# 5db13cd = 1
# 5d80c9a = 1
# 022dccf = - 1

In [27]:
# Fix the beat numbers
for index in range(len(beats)):
    beats[index][1] = int((index - 1) % 8)
beats

array([[0.22, 7.  ],
       [0.66, 0.  ],
       [1.14, 1.  ],
       [1.58, 2.  ],
       [2.04, 3.  ],
       [2.52, 4.  ],
       [2.98, 5.  ],
       [3.44, 6.  ],
       [3.9 , 7.  ],
       [4.38, 0.  ],
       [4.84, 1.  ],
       [5.3 , 2.  ],
       [5.78, 3.  ],
       [6.24, 4.  ],
       [6.7 , 5.  ],
       [7.16, 6.  ],
       [7.62, 7.  ],
       [8.08, 0.  ]])

In [9]:
'''
beat_start = 0.66
beat_end = 8.2
num_beats = 16
beat_delta = (beat_end - beat_start) / (num_beats - 1)
beats = [[round(beat_start + x * beat_delta, 2), x % 8] for x in range(num_beats)]
beats
'''

'\nbeat_start = 0.66\nbeat_end = 8.2\nnum_beats = 16\nbeat_delta = (beat_end - beat_start) / (num_beats - 1)\nbeats = [[round(beat_start + x * beat_delta, 2), x % 8] for x in range(num_beats)]\nbeats\n'

In [11]:
'''
# Fix the beat numbers
for index in range(12, 16):
    beats[index][1] = beats[index][1] - 4
beats
'''

'\n# Fix the beat numbers\nfor index in range(12, 16):\n    beats[index][1] = beats[index][1] - 4\nbeats\n'

In [35]:
from moviepy import VideoFileClip, TextClip, CompositeVideoClip

video_file = f"inputs/videos/{move_filename}.mp4"
output_file = f"processed/videos/{move_filename}_counts.mp4"


# Load the video
video_clip = VideoFileClip(video_file)

start_counting = True
text_clips = [video_clip]

for index, x in enumerate(beats):
    start_time = x[0]
    end_time = beats[index + 1][0] if index + 1 < len(beats) else x[0]

    start_counting = start_counting or x[1] == 1

    if start_counting:
        # Create the text clip
        text_clip = TextClip(font='arial.ttf', text=f'{int(x[1] + 1)}', font_size=70, color='white', duration=end_time - start_time)
        
        # Set the duration and position of the text clip
        text_clip = text_clip.with_position(('center', 'bottom'))
        
        # Set the start time of the text clip
        text_clip = text_clip.with_start(start_time)

        text_clips.append(text_clip)
        
# Composite the text clip on the video
video_clip = CompositeVideoClip(text_clips)

# Write the output video file
video_clip.write_videofile(output_file, fps=30, codec='libx264')

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf62.3.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 127, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'video', 'language': None, 'default': True, 'size': [720, 1280], 'bitrate': 9512, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc62.11.100 libx264'}}], 'input_number': 0}], 'duration': 8.37, 'bitrate': 9651, 'start': 0.0, 'default_audio_input_number': 0, 'default_audio_stream_number': 0, 'audio_fps': 44100, 'audio_bitrate': 127, 'default_video_input_number': 0, 'default_video_stream_number': 1, 'video_codec_na

MoviePy - Done.
MoviePy - Writing video processed/videos/022dccf_counts.mp4



MoviePy - Done !
MoviePy - video ready processed/videos/022dccf_counts.mp4


In [36]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

# After verifying that the counts are Okay, I can save the counts
data[move_filename] = []
section = 1
for index, x in enumerate(beats):
    count = int(x[1] + 1)
    print(count)
    data[move_filename].append({
        'section': section, 
        'count': count,
        'time': x[0],
        'leader_keypoints': [],
        'follower_keypoints': [],
        'offset': offset
    })
    if count == 8:
        section += 1

with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

8
1
2
3
4
5
6
7
8
1
2
3
4
5
6
7
8
1


In [43]:
import cv2

start_counting = False
section = 1
for index, x in enumerate(beats):
    count = int(x[1] + 1)
    print(count)
    vidcap = cv2.VideoCapture(video_file)
    vidcap.set(cv2.CAP_PROP_POS_MSEC, int(x[0] * 1000))      # just cue to position
    success, image = vidcap.read()
    if success:
        cv2.imwrite(os.path.join('processed', 'images', f"{move_filename}_section_{section}_count_{count}.jpg"), image)     # save frame as JPEG file

    # Release resources
    vidcap.release()
    cv2.destroyAllWindows()

    if count == 8 or (count == 4 and index < len(beats) - 2 and beats[index + 1][1] == 0):
        section += 1

8
1
2
3
4
5
6
7
8
1
2
3
4
5
6
7
8
1


In [47]:
import numpy as np
import cv2
import os
from rtmlib import Body, draw_skeleton
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

device = 'cpu'
backend = 'onnxruntime'  # opencv, onnxruntime, openvino

cap = cv2.VideoCapture(0)

openpose_skeleton = False  # True for openpose-style, False for mmpose-style

body = Body(
    pose='rtmo',
    to_openpose=openpose_skeleton,
    mode='performance',  # balanced, performance, lightweight
    backend=backend,
    device=device
)
# TODO: Nowadays, I do this by hand. We need to automate it.
leader_detected_first = [
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    True,
    False,
    False,
    False,
    True,
    False,
    True,
    False,
    False,
]
#leader_detected_first = [False for index in range(len(beats))]
img_index = 0

for filename in os.listdir(os.path.join('processed', 'images')):
    # check if the image ends with png
    if (filename.startswith(move_filename)):
        frame = cv2.imread(os.path.join('processed', 'images', filename), cv2.IMREAD_COLOR)
        keypoints, scores = body(frame)

        if leader_detected_first[img_index]:
            data[move_filename][img_index]["leader_keypoints"] = keypoints[0].tolist()
            if keypoints.shape[0] > 1:
                data[move_filename][img_index]["follower_keypoints"] = keypoints[1].tolist()
        elif np.sum(keypoints) > 0:
            data[move_filename][img_index]["follower_keypoints"] = keypoints[0].tolist()
            if keypoints.shape[0] > 1:
                data[move_filename][img_index]["leader_keypoints"] = keypoints[1].tolist()
        print(filename)
        print(keypoints.shape)
        print(np.sum(keypoints))
        print(keypoints[0])

        img_show = frame.copy()
        img_show = draw_skeleton(
            img_show,
            keypoints,
            #np.array([keypoints[0]]),
            scores,
            openpose_skeleton=openpose_skeleton,
            kpt_thr=0.3,
            line_width=2
        )

        cv2.imwrite(os.path.join('processed', 'keypoints', filename), img_show)
        img_index += 1

# Override the data with the keypoints
with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

load C:\Users\user\.cache\rtmlib\hub\checkpoints\rtmo-l_16xb16-600e_body7-640x640-b37118ce_20231211.onnx with onnxruntime backend
022dccf_section_1_count_8.jpg
(1, 17, 2)
16523.64
[[ 373.758    413.17023]
 [ 372.62753  403.95584]
 [ 369.13837  403.87195]
 [ 326.8739   421.60345]
 [ 382.63174  421.87936]
 [ 280.35242  506.0938 ]
 [ 434.49274  511.43677]
 [ 174.34074  492.8554 ]
 [ 494.3369   623.8365 ]
 [ 136.54486  417.60736]
 [ 537.1737   615.5463 ]
 [ 285.33795  737.745  ]
 [ 370.5668   741.58374]
 [ 323.5166   908.66345]
 [ 376.6397   912.5169 ]
 [ 285.09702 1072.0354 ]
 [ 322.78326 1073.0265 ]]
022dccf_section_2_count_1.jpg
(1, 17, 2)
17438.125
[[ 350.7252   455.37158]
 [ 349.51486  443.04443]
 [ 361.09467  446.92157]
 [ 319.0607   454.52078]
 [ 383.5799   459.03156]
 [ 268.5423   533.1891 ]
 [ 413.7632   530.6408 ]
 [ 255.05936  667.2087 ]
 [ 453.36017  649.72015]
 [ 203.93271  690.14844]
 [ 526.2915   684.14166]
 [ 307.34988  754.06305]
 [ 386.44034  759.7526 ]
 [ 219.2431   905.

In [49]:
import numpy as np

def normalize_keypoints_by_bbox_height(keypoints):
    """
    Normalize keypoints so that:
    - The top-left of the bounding box is (0, 0)
    - The bounding box height is 1
    """
    kp = np.array(keypoints)
    
    min_x = np.min(kp[:, 0])
    min_y = np.min(kp[:, 1])
    max_y = np.max(kp[:, 1])
    
    bbox_height = max_y - min_y
    if bbox_height < 1e-5:
        raise ValueError("Bounding box height too small for normalization.")
    
    # Translate to origin and scale
    kp_normalized = (kp - [min_x, min_y]) / bbox_height
    return kp_normalized, bbox_height


def compute_normalized_embeddings(leader_kp, follower_kp):
    leader_kp = np.array(leader_kp)
    follower_kp = np.array(follower_kp)

    # Normalize each person's keypoints by their own bbox height
    norm_leader = np.array([])
    norm_follower = np.array([])
    if np.sum(leader_kp) > 0:
        norm_leader, leader_height = normalize_keypoints_by_bbox_height(leader_kp)
    if np.sum(follower_kp) > 0:
        norm_follower, _ = normalize_keypoints_by_bbox_height(follower_kp)

    rel_vector = np.array([])
    if np.sum(leader_kp) > 0 and np.sum(follower_kp) > 0:
        # Get original centers (before normalization)
        center_leader = np.mean(leader_kp, axis=0)
        center_follower = np.mean(follower_kp, axis=0)
    
        # Compute relative position vector in leader's height units
        rel_vector = (center_follower - center_leader) / leader_height

    return norm_leader.flatten(), norm_follower.flatten(), rel_vector

In [51]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

for index in range(len(data[move_filename])):
    kp_leader = data[move_filename][index]['leader_keypoints']
    kp_follower = data[move_filename][index]['follower_keypoints']
    leader_embedding, follower_embedding, distance_embedding = compute_normalized_embeddings(kp_leader, kp_follower)
    data[move_filename][index]['leader_embedding'] = leader_embedding.tolist()
    data[move_filename][index]['follower_embedding'] = follower_embedding.tolist()
    data[move_filename][index]['distance_embedding'] = distance_embedding.tolist()

# Override the data with the new embeddings
with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

In [163]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

# Add musicality (TODO: This is manual)
musicality = [
    '-',
    '-',
    '-',
    '-',
    '-',
    '-',
    '-',
    '-',
    '-',
    '*',
    '-',
    '-',
    '-',
    '*',
    '-',
    '-',
    '-',
    '-'
]
for index in range(len(data[move_filename])):
    data[move_filename][index]['musicality'] = musicality[index]

# Override the data with musicality
with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)